# Objectif : Comparer les produits structurés, effectuer du clustering pour identifier les outliers

Nous allons faire un embedding des descriptions de produits structurés ainsi que des caractéristiques que nous avons scrappé pour déterminer un score de similarité. Ensuite, nous effectuerons des algorithmes de clustering sur ce score pour déterminer qui sont les outliers, sous-entendu les produits structurés complexes.

Pour cela, il faudra faire apparaitre dans le résumé le détail du fonctionnement du produit structuré afin de pouvoir vraiment comparer.

In [3]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [6]:
df=pd.read_csv(os.path.join(os.getcwd(),'output/bdd_DIC.csv')) 

print(df.columns)

Index(['code_ISIN', 'nom_du_produit', 'emetteur_du_produit', 'date_emission',
       'date_remboursement', 'mention_complexite',
       'montant_minimum_investissement', 'niveau_garantie',
       'niveau_barriere_desactivante', 'niveau_risque', 'produit_sous_jacent',
       'nature_sous_jacent', 'code_ISIN_sous_jacent', 'frais_ponctuels_entree',
       'frais_ponctuels_sortie', 'frais_recurrents', 'frais_accessoires',
       'performance_tension', 'performance_maximale',
       'espérance_maximale_rendement'],
      dtype='object')


In [ ]:

pd.options.mode.chained_assignment = None 
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["text"].apply(lambda x: len(tokenizer.encode(x)))

print(len(df))
df = df[df.n_tokens<8192]
print(len(df))

In [ ]:
os.environ["AZURE_OPENAI_KEY"] = "OPENAI_API_KEY"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
os.environ["AZURE_OPENAI_DeploymentId"] = "ada-002"
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)